In [1]:
pip install mysql-connector-python Pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector

def create_db_connection():
    return mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='color_db'
    )

In [2]:
from PIL import Image
import io

def upload_image(image_path):
    conn = create_db_connection()
    cursor = conn.cursor()
    
    # 이미지 열기 및 속성 얻기
    with Image.open(image_path) as img:
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=img.format)
        img_bytes = img_byte_arr.getvalue()
        
        # 이미지 속성
        file_name = image_path.split('/')[-1]
        extension = img.format
        file_size = len(img_bytes)
        width, height = img.size
    
    # SQL 쿼리 준비
    sql = """INSERT INTO blob_image (f_id, fname, extname, fsize, f_width, f_height, f_data) 
             VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    values = (1, file_name, extension, file_size, width, height, img_bytes)
    
    # 실행 및 커밋
    cursor.execute(sql, values)
    conn.commit()
    cursor.close()
    conn.close()

    print("이미지가 성공적으로 업로드 되었습니다!")


In [3]:
import os

def download_image(file_id, output_path):
    conn = create_db_connection()
    cursor = conn.cursor()
    
    # 이미지 검색을 위한 SQL
    sql = "SELECT fname, extname, f_data FROM blob_image WHERE f_id = %s"
    cursor.execute(sql, (file_id,))
    file_name, extension, img_data = cursor.fetchone()
    
    # 다운로드할 경로 확인 및 생성
    if not os.path.exists(output_path):
        os.makedirs(output_path)  # 경로가 없으면 생성
    
    # 파일로 이미지 데이터 쓰기
    full_path = os.path.join(output_path, file_name)
    with open(full_path, 'wb') as f:
        f.write(img_data)
    
    cursor.close()
    conn.close()

    print(f"{output_path}/{file_name}에 이미지가 성공적으로 다운로드 되었습니다.")


In [5]:
image_to_upload = ""  # 업로드할 이미지의 경로
upload_image(image_to_upload)

이미지가 성공적으로 업로드 되었습니다!


In [6]:
file_id_to_download = 1  # 데이터베이스에서 다운로드할 이미지의 파일 ID
download_path = ""  # 이미지를 저장할 경로
download_image(file_id_to_download, download_path)

downloadimage/testimage.jpg에 이미지가 성공적으로 다운로드 되었습니다.
